In [1]:
################################
#Packages
using Gurobi
using JuMP
using Plots
using GLMNet
using DataFrames 
using CSV
using Statistics
using Random


### Read in Data

In [13]:
impute_filepath = "Data/data_preimpute.csv"

data = CSV.read(impute_filepath,DataFrame)
allowmissing!(data)
;

In [14]:
first(data,10)

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Years_Opened,Item_Type_Baking Goods
,Float64?,Float64?,Float64?,Float64?,Int64?,Int64?
1,9.3,0.0160473,249.809,3735.14,14,0
2,5.92,0.0192782,48.2692,443.423,4,0
3,17.5,0.0167601,141.618,2097.27,14,0
4,19.2,9999.0,182.095,732.38,15,0
5,8.93,9999.0,53.8614,994.705,26,0
6,10.395,9999.0,51.4008,556.609,4,1
7,13.65,0.0127411,57.6588,343.553,26,0
8,9999.0,0.12747,107.762,4022.76,28,0
9,16.2,0.0166871,96.9726,1076.6,11,0


## Set the missing values to "missing"

In [15]:
for i = 1:nrow(data)
    if data[i, :Item_Visibility] == 9999
        data[i,:Item_Visibility] = missing
        
    end
    
    if data[i, :Item_Weight] == 9999
        data[i,:Item_Weight] = missing
        
    end
    
    
end

In [17]:
first(data,10)

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Years_Opened,Item_Type_Baking Goods
,Float64?,Float64?,Float64?,Float64?,Int64?,Int64?
1,9.3,0.0160473,249.809,3735.14,14,0
2,5.92,0.0192782,48.2692,443.423,4,0
3,17.5,0.0167601,141.618,2097.27,14,0
4,19.2,missing,182.095,732.38,15,0
5,8.93,missing,53.8614,994.705,26,0
6,10.395,missing,51.4008,556.609,4,1
7,13.65,0.0127411,57.6588,343.553,26,0
8,missing,0.12747,107.762,4022.76,28,0
9,16.2,0.0166871,96.9726,1076.6,11,0


### Implement Optimal Impute

In [21]:
data_imputed = IAI.impute(data)

In [18]:
names(data)[20:35]

16-element Vector{String}:
 "Item_Type_Starchy Foods"
 "Item_Fat_Content_Regular"
 "Outlet_Identifier_OUT013"
 "Outlet_Identifier_OUT017"
 "Outlet_Identifier_OUT018"
 "Outlet_Identifier_OUT019"
 "Outlet_Identifier_OUT027"
 "Outlet_Identifier_OUT035"
 "Outlet_Identifier_OUT045"
 "Outlet_Identifier_OUT046"
 "Outlet_Identifier_OUT049"
 "Outlet_Location_Type_Tier 2"
 "Outlet_Location_Type_Tier 3"
 "Outlet_Type_Supermarket Type1"
 "Outlet_Type_Supermarket Type2"
 "Outlet_Type_Supermarket Type3"

### Asses how imputation did (needs more work)

In [19]:
store_13_idx = findall(x->x ==1, data[:, :Outlet_Identifier_OUT013]);
store_13 = data_imputed[store_13_idx,:]
store_13_Vis = store_13[:,:Item_Visibility]
sum(store_13_Vis)

LoadError: UndefVarError: data_imputed not defined

In [ ]:
store_13_idx = findall(x->x ==1, data[:, :Outlet_Identifier_OUT046]);
store_13 = data_imputed[store_13_idx,:]
store_13_Vis = store_13[:,:Item_Visibility]
sum(store_13_Vis)

In [ ]:
CSV.write("Data/Optimal_Impute_Data.csv", data_imputed)